# Astropy: Tables


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Create tables</li>
<li>Access data in tables</li>
<li>Combining tables</li>
<li>Aggregation</li>
<li>Masking</li>
<li>Reading/writing</li>
</ul>

</div>

</section>


## Documentation

For more information about the features presented below, you can read the
[astropy.table](http://docs.astropy.org/en/stable/table/index.html) docs.

## Creating tables

In [2]:
from astropy.table import Table
import numpy as np

In [7]:
t1 = Table()
t1['name'] = ['source 1', 'source 2', 'source 3']
t1['flux'] = [1.2, 2.2, 3.1]
print(t1)

  name   flux
-------- ----
source 1  1.2
source 2  2.2
source 3  3.1


In [13]:
t1

name,flux,size
str8,float64,int64
source 1,1.2,1
source 2,2.2,5
source 3,3.1,4


In [12]:
t1['size'] = [1,5,4]

In [16]:
t1['size']

1
5
4


In [17]:
np.array(t1['size'])

array([1, 5, 4])

In [22]:
t1[0]

name,flux,size
str8,float64,int64
source 1,1.2,1


In [23]:
t1[0]['size']

1

## Units in tables

In [37]:
import astropy.units as u
t1['size'].unit = 'cm'
t1

name,flux,size
,,cm
str8,float64,int64
source 1,1.2,1
source 2,2.2,5
source 3,3.1,4


Some unitful operations will then work:

In [38]:
t1['size'].to('m')

<Quantity [0.01, 0.05, 0.04] m>

However, you may run into unexpected behavior, so if you are planning on using table columns as Quantities, we recommend that you use the ``QTable`` class:

In [39]:
type(t1['size'])

astropy.table.column.Column

In [42]:
from astropy.table import QTable
qt1 = QTable(t1)
type(qt1['size'])

astropy.units.quantity.Quantity


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<ol>
<li>Make a table that contains three columns: <code>spectral type</code>, <code>temperature</code>, and <code>radius</code>, and incude 5 rows with fake data (or real data if you like, for example from <a href="http://www.atlasoftheuniverse.com/startype.html">here</a>). Try including units on the columns that can have them.</li>
<li>Find the mean temperature and the maximum radius</li>
<li>Try and find out how to add and remove rows</li>
<li>Add a new column which gives the luminosity (using $L=4\pi R^2 \sigma T^4$)</li>
</ol>

</div>

</section>


In [138]:
import astropy.units as u
import astropy.constants as const

myTab = QTable()
myTab['spectral type'] = ['O', 'B', 'A', 'F', 'G']
myTab['temperature'] = [50000, 20000, 15000, 10000, 8000]*u.K
myTab['radius'] = [20000, 10000, 5000, 2000, 1000]*u.km #u.R_sun

print(myTab['temperature'].mean())
print(myTab['radius'].max())

#myTab.insert_row(5, ['K', 5000, 5000])
myTab.add_row(['K', 5000*u.K, 5000*u.km])
#myTab.remove_row(2)
lumi = 4*np.pi*myTab['radius']**2*myTab['temperature']**4*const.sigma_sb#*u.Watt*u.K**-4*u.m**-2
myTab['luminosity'] = lumi.decompose().to(u.Watt)
#myTab['luminosity'].unit = u.watt
# damit string länger wird!
# spalte löschen und neu hinzufügen mit str2
myTab

20600.0 K
20000.0 km


spectral type,temperature,radius,luminosity
,K,km,W
str1,float64,float64,float64
O,50000.0,20000.0,1.7813983310357992e+27
B,20000.0,10000.0,1.1400949318629114e+25
A,15000.0,5000.0,9.018329050868734e+23
F,10000.0,2000.0,2.850237329657279e+22
G,8000.0,1000.0,2.9186430255690537e+21
K,5000.0,5000.0,1.1133739568973744e+22


## Iterating over tables

It is possible to iterate over rows or over columns. To iterate over rows, simply iterate over the table itself:

In [142]:
for row in t1:
    print(row['name'])

source 1
source 2
source 3


Rows can act like dictionaries, so you can access specific columns from a row:

In [148]:
for colName in t1.colnames:
        print(t1[colName][1])
        
print(t1['name'])

source 2
2.2
5
  name  
--------
source 1
source 2
source 3


Iterating over columns is also easy:

In [153]:
t1

name,flux,size
,,cm
str8,float64,int64
source 1,1.2,1
source 2,2.2,5
source 3,3.1,4


Accessing specific rows from a column object can also be done with the item notation:

## Joining tables

In [149]:
from astropy.table import join

In [152]:
t2 = Table()
t2['name'] = ['source 1', 'source 3']
t2['flux2'] = [1,9]
t2

name,flux2
str8,int64
source 1,1
source 3,9


In [156]:
t3 = join(t1, t2, join_type='outer')
t3

name,flux,size,flux2
,,cm,
str8,float64,int64,int64
source 1,1.2,1,1
source 2,2.2,5,--
source 3,3.1,4,9


## Masked tables

In [182]:
t3['flux2'].mask[2] = False
t3
#np.min(t3['flux2'])

name,flux,size,flux2
,,cm,
str8,float64,int64,int64
source 1,1.2,1,1
source 2,2.2,5,1
source 3,3.1,4,9


## Slicing

Tables can be sliced like Numpy arrays:

In [176]:
obs = Table()
obs['name'] = ['M31', 'M32', 'M31', 'M40', 'M50']
obs['mag'] = [4, 6, 8, 7, 6]

t1['flux'][0]

1.2

In [189]:
obs[0:4]

name,mag
str3,int64
M31,4
M32,6
M31,8
M40,7


In [192]:
obs[obs['mag'] > 6]

name,mag
str3,int64
M31,8
M40,7


In [195]:
obs['mag', 'name']

mag,name
int64,str3
4,M31
6,M32
8,M31
7,M40
6,M50



<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Starting from the <code>obs</code> table:</p>
<ol>
<li>Make a new table that shows every other row, starting with the second row? (that is, the second, fourth, sixth, etc. rows).</li>
<li>Make a new table the only contains rows where <code>name</code> is <code>M31</code></li>
</ol>

</div>

</section>


In [201]:
obs[1::2]

name,mag
str3,int64
M32,6
M40,7


In [198]:
obs[obs['name'] == 'M31']

name,mag
str3,int64
M31,4
M31,8


## Grouping and Aggregation

In [207]:
obs_by_name = obs.group_by('name')
for group in obs_by_name.groups:
    print(group)
    print('')

name mag
---- ---
 M31   4
 M31   8

name mag
---- ---
 M32   6

name mag
---- ---
 M40   7

name mag
---- ---
 M50   6



In [210]:
obs_by_name.groups.aggregate(np.mean)

name,mag
str3,float64
M31,6.0
M32,6.0
M40,7.0
M50,6.0


## Writing data

In [211]:
obs.write('obstable.fits')

In [222]:
obs.write('obstable.vot', format='votable')

OSError: File exists: obstable.vot

In [223]:
!cat obstable.vot

<?xml version="1.0" encoding="utf-8"?>
<!-- Produced with astropy.io.votable version 3.0.4
     http://www.astropy.org/ -->
<VOTABLE version="1.3" xmlns="http://www.ivoa.net/xml/VOTable/v1.3" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://www.ivoa.net/xml/VOTable/v1.3">
 <RESOURCE type="results">
  <TABLE>
   <FIELD ID="name" arraysize="3" datatype="unicodeChar" name="name"/>
   <FIELD ID="mag" datatype="long" name="mag"/>
   <DATA>
    <TABLEDATA>
     <TR>
      <TD>M31</TD>
      <TD>4</TD>
     </TR>
     <TR>
      <TD>M32</TD>
      <TD>6</TD>
     </TR>
     <TR>
      <TD>M31</TD>
      <TD>8</TD>
     </TR>
     <TR>
      <TD>M40</TD>
      <TD>7</TD>
     </TR>
     <TR>
      <TD>M50</TD>
      <TD>6</TD>
     </TR>
    </TABLEDATA>
   </DATA>
  </TABLE>
 </RESOURCE>
</VOTABLE>


## Reading data

In [226]:
t4=Table.read('2mass.tbl', format='ascii.ipac')
t4

ra,dec,clon,clat,err_maj,err_min,err_ang,designation,j_m,j_cmsig,j_msigcom,j_snr,h_m,h_cmsig,h_msigcom,h_snr,k_m,k_cmsig,k_msigcom,k_snr,ph_qual,rd_flg,bl_flg,cc_flg,ndet,gal_contam,mp_flg,dist,angle,j_h,h_k,j_k
deg,deg,,,arcsec,arcsec,deg,,mag,mag,mag,,mag,mag,mag,,mag,mag,mag,,,,,,,,,,,,,
float64,float64,str12,str13,float64,float64,int64,str16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str3,str3,str3,str3,str6,int64,int64,float64,float64,float64,float64,float64
274.429506,-13.870547,18h17m43.08s,-13d52m13.97s,0.08,0.08,45,18174308-1352139,16.305,0.142,0.143,6.7,14.048,0.107,0.108,13.6,13.257,0.066,0.066,16.5,CAA,222,111,0ss,066655,0,0,975.080151,256.448,2.257,0.791,3.048
274.423821,-13.86974,18h17m41.72s,-13d52m11.06s,0.06,0.06,90,18174171-1352110,14.802,0.058,0.059,26.7,12.635,0.059,0.06,50.1,11.768,0.045,0.046,65.2,AAA,222,111,0ss,666666,0,0,993.752042,256.878,2.167,0.867,3.034
274.424587,-13.739629,18h17m41.90s,-13d44m22.66s,0.08,0.08,45,18174190-1344226,16.328,--,--,--,14.345,0.059,0.06,10.4,13.405,0.046,0.047,14.4,UAA,022,011,0cc,003666,0,0,995.726698,284.113,--,0.94,--
274.433933,-13.769502,18h17m44.14s,-13d46m10.21s,0.08,0.08,45,18174414-1346102,16.281,0.098,0.099,6.8,14.057,0.035,0.036,13.5,12.956,0.032,0.033,21.8,CAA,222,111,000,065566,0,0,942.627418,278.252,2.224,1.101,3.325
274.437013,-13.885698,18h17m44.88s,-13d53m08.51s,0.09,0.09,45,18174488-1353085,15.171,--,--,--,14.412,0.152,0.152,9.8,13.742,0.095,0.095,10.6,UBA,622,022,0cc,005566,0,0,964.105389,252.93,--,0.67,--
274.433996,-13.752446,18h17m44.16s,-13d45m08.81s,0.08,0.08,90,18174415-1345088,16.54,--,--,--,14.519,0.083,0.083,8.8,13.604,0.043,0.044,12.0,UBA,022,011,0cc,005666,0,0,953.230532,281.908,--,0.915,--
274.418138,-13.77215,18h17m40.35s,-13d46m19.74s,0.08,0.08,90,18174035-1346197,17.98,--,--,--,14.61,0.043,0.044,8.1,13.456,0.056,0.057,13.8,UBA,022,011,000,001645,0,0,996.047248,277.25,--,1.154,--
274.433695,-13.899049,18h17m44.09s,-13d53m56.58s,0.06,0.06,90,18174408-1353565,13.011,0.021,0.024,139.0,10.917,0.02,0.021,243.8,10.013,0.017,0.019,328.3,AAA,222,111,000,666666,0,0,990.166399,250.466,2.094,0.904,2.998



<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge</h2>
</div>


<div class="panel-body">

<p>Using the <code>t4</code> table above:</p>
<ol>
<li>
<p>Make a plot that shows <code>j_m</code>-<code>h_m</code> on the x-axis, and <code>h_m</code>-<code>k_m</code> on the y-axis</p>
</li>
<li>
<p>Make a new table that contains the subset of rows where the <code>j_snr</code>, <code>h_snr</code>, and <code>k_snr</code> columns, which give the signal-to-noise-ratio in the J, H, and K band, are greater than 10, and try and show these points in red in the plot you just made.</p>
</li>
<li>
<p>Make a new table (based on the full table) that contains only the RA, Dec, and the <code>j_m</code>, <code>h_m</code> and <code>k_m</code> columns, then try and write out this catalog into a format that you can read into another software package. For example, try and write out the catalog into CSV format, then read it into a spreadsheet software package (e.g. Excel, Google Docs, Numbers, OpenOffice). You may run into an issue at this point - if so, take a look at https://github.com/astropy/astropy/issues/7357 to see how to fix it.</p>
</li>
</ol>

</div>

</section>
